In [3]:
import pandas as pd
import numpy as np

In [2]:
df_all = pd.read_csv('../data/all_measurements.csv')

# Experiment 01

In [4]:
def boxplot_stats(series):
    q1 = np.percentile(series, 25)
    q3 = np.percentile(series, 75)
    median = np.median(series)
    iqr = q3 - q1
    lower_whisker = max(series[series >= q1 - 1.5 * iqr].min(), series.min())
    upper_whisker = min(series[series <= q3 + 1.5 * iqr].max(), series.max())
    return lower_whisker, q1, median, q3, upper_whisker

In [5]:
df_exp1_client = df_all[
    (df_all["experiment"] == "experiment_01") &
    (df_all["side"] == "client")
    ].copy()

In [6]:
modes = df_exp1_client['mode'].unique()

In [7]:
colors = ["blue!20", "orange!20", "green!20"]

In [13]:
print(r"\begin{axis}[")
print(r"  boxplot/draw direction=y,")
print(r"  ylabel={Handshake Duration (s)},")
print(r"  xlabel={Key Exchange Mode},")
print(r"  xtick={1,2,3},")
print(r"  xticklabels={Classic, PQC, Hybrid},")
print(r"  ymode=log,")
print(r"  width=0.85\linewidth,")
print(r"  height=7cm,")
print(r"  grid=major")
print(r"]")

for idx, mode in enumerate(modes, start=1):
    durations = df_exp1_client[df_exp1_client['mode'] == mode]['duration_sec']
    lw, q1, med, q3, uw = boxplot_stats(durations)
    print(rf"\addplot+[" + "\n"
          rf"  boxplot prepared={{" + "\n"
          rf"    lower whisker={lw:.6f}," + "\n"
          rf"    lower quartile={q1:.6f}," + "\n"
          rf"    median={med:.6f}," + "\n"
          rf"    upper quartile={q3:.6f}," + "\n"
          rf"    upper whisker={uw:.6f}" + "\n"
          rf"  }}," + "\n"
          rf"  fill={colors[idx-1]}" + "\n"
          rf"] coordinates {{}};")

print(r"\end{axis}")


\begin{axis}[
  boxplot/draw direction=y,
  ylabel={Handshake Duration (s)},
  xlabel={Key Exchange Mode},
  xtick={1,2,3},
  xticklabels={Classic, PQC, Hybrid},
  ymode=log,
  width=0.85\linewidth,
  height=7cm,
  grid=major
]
\addplot+[
  boxplot prepared={
    lower whisker=0.000162,
    lower quartile=0.000226,
    median=0.000245,
    upper quartile=0.000268,
    upper whisker=0.000331
  },
  fill=blue!20
] coordinates {};
\addplot+[
  boxplot prepared={
    lower whisker=0.000066,
    lower quartile=0.000098,
    median=0.000155,
    upper quartile=0.000206,
    upper whisker=0.000360
  },
  fill=orange!20
] coordinates {};
\addplot+[
  boxplot prepared={
    lower whisker=0.000223,
    lower quartile=0.000283,
    median=0.000304,
    upper quartile=0.000324,
    upper whisker=0.000386
  },
  fill=green!20
] coordinates {};
\end{axis}


In [19]:
# Filter z. B. nur classic
df_classic = df_exp1_client[df_exp1_client['mode'] == 'hybrid']
durations = df_classic['duration_sec']

# IQR-Logik
q1 = np.percentile(durations, 25)
q3 = np.percentile(durations, 75)
iqr = q3 - q1
lw = max(durations[durations >= q1 - 1.5 * iqr].min(), durations.min())
uw = min(durations[durations <= q3 + 1.5 * iqr].max(), durations.max())

# ➤ Nur die Ausreißer:
outliers = durations[(durations < lw) | (durations > uw)]

# Für pgfplots vorbereiten
for val in outliers:
    print(f"(1,{val:.6f})")

(1,0.001789)
(1,0.000456)
(1,0.000507)
(1,0.000395)
(1,0.000400)
(1,0.000449)
(1,0.000490)
(1,0.000428)
(1,0.000609)
(1,0.000583)
(1,0.000488)
(1,0.000429)
(1,0.000411)
(1,0.000411)
(1,0.000428)
(1,0.000396)
(1,0.000391)
(1,0.000388)
(1,0.000407)
(1,0.000414)
(1,0.000415)
(1,0.000565)
(1,0.000390)
(1,0.000451)
(1,0.000388)
(1,0.000396)
(1,0.000392)
(1,0.000412)
(1,0.000400)
(1,0.000449)
(1,0.000444)
(1,0.000571)
(1,0.001170)
(1,0.000552)
(1,0.000432)
(1,0.000422)
(1,0.000428)
(1,0.000388)
(1,0.000406)
(1,0.000408)
(1,0.000442)
(1,0.000405)
(1,0.000397)
(1,0.000388)
(1,0.000415)
(1,0.000415)
(1,0.000404)
(1,0.000392)
(1,0.000398)
(1,0.000416)
(1,0.000408)
(1,0.000399)
(1,0.000386)
(1,0.000411)
(1,0.000401)
(1,0.000399)
(1,0.000403)
(1,0.000391)
(1,0.000395)
(1,0.000406)
(1,0.000404)
(1,0.000391)
(1,0.000389)
(1,0.000416)
(1,0.000409)
(1,0.000407)
(1,0.000413)
(1,0.000396)
(1,0.000399)
(1,0.000387)
(1,0.000394)
(1,0.000394)
(1,0.000451)
(1,0.000396)
(1,0.000388)
(1,0.000406)
(1,0.000396)